In [13]:
import requests
import pandas as pd
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support import expected_conditions as EC

def scrape_imoveis(num_clicks=5):
    opts = Options()
    opts.add_argument("--headless")  # Modo headless para reduzir uso de memória
    opts.add_argument("--disable-dev-shm-usage")  # Evitar problemas de memória
    opts.add_argument("--no-sandbox")  # Evitar problemas de memória

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=opts
    )

    lista_de_imoveis = []

    try:
        driver.get('https://www.thaisimobiliaria.com.br/imoveis/a-venda')

        # Remover o elemento de cookies usando JavaScript
        driver.execute_script("document.getElementById('cookies-component').remove();")

        sleep(3)

        # Calcular o número mínimo de cliques necessários
        total_imoveis_desejado = 3750
        imoveis_por_pagina = 12
        num_paginas_necessarias = (total_imoveis_desejado - 1) // imoveis_por_pagina + 1
        num_clicks = min(num_clicks, num_paginas_necessarias)

        for click_number in range(num_clicks):
            try:
                # Esperar até que o botão "Ver Mais" esteja visível
                botao_ver_mais = WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.CSS_SELECTOR, '.btn.btn-md.btn-primary.btn-next'))
                )
                botao_ver_mais.click()
                sleep(0.5)  # Reduzir o tempo de espera entre os cliques

                print(f"Clique número {click_number + 1}")

            except (NoSuchElementException, TimeoutException) as e:
                print(f"Erro ao clicar no botão 'Ver Mais': {e}")
                break

        page_content = driver.page_source
        site = BeautifulSoup(page_content, 'html.parser')
        imoveis = site.find_all('a', attrs={'class': 'card_split_vertically borderHover'})

        for imovel in imoveis:
            titulo = imovel.find('h2', attrs={'class': 'card_split_vertically__location'})
            titulo_text = titulo.text.strip() if titulo else None

            link = 'https://www.thaisimobiliaria.com.br' + imovel['href']

            tipo = imovel.find('p', attrs={'class': 'card_split_vertically__type'})
            tipo_text = tipo.text.strip() if tipo else None

            preco_area = imovel.find('div', attrs={'class': 'card_split_vertically__value-container'})
            preco = preco_area.find('p', attrs={'class': 'card_split_vertically__value'}).text.strip() if preco_area else "Preço não especificado"
            preco = ''.join(filter(str.isdigit, preco))  # Remover caracteres não numéricos

            if not preco or preco == '0':  # Verificar se o preço está ausente ou igual a zero
                continue  # Ignorar este imóvel e passar para o próximo
            
            metro = imovel.find('li', attrs={'class': 'card_split_vertically__spec'})
            metro_text = metro.text.replace('m²', '').strip() if metro else None
            metro_text = ''.join(filter(str.isdigit, metro_text))  # Remover caracteres não numéricos

            # quartos, suíte, banheiros, vagas
            quarto_suite_banheiro_vaga = imovel.find('ul', attrs={'class': 'card_split_vertically__specs'})
            if quarto_suite_banheiro_vaga:
                lista = quarto_suite_banheiro_vaga.findAll('li')
                quarto = suite = banheiro = vaga = 0  # Valor padrão 0 para substituir espaços em branco

                for item in lista:
                    text_lower = item.text.lower()
                    if 'quarto' in text_lower or 'quartos' in text_lower:
                        quarto = int(item.text.split()[0])  # Apenas o primeiro número
                    elif 'suíte' in text_lower or 'suítes' in text_lower:
                        suite = int(item.text.split()[0])  # Apenas o primeiro número
                    elif 'banheiro' in text_lower or 'banheiros' in text_lower:
                        banheiro = int(item.text.split()[0])  # Apenas o primeiro número
                    elif 'vaga' in text_lower or 'vagas' in text_lower:
                        vaga = int(item.text.split()[0])  # Apenas o primeiro número
            else:
                quarto = suite = banheiro = vaga = 0

            lista_de_imoveis.append([titulo_text, tipo_text, link, preco, metro_text, quarto, suite, banheiro, vaga])

    finally:
        driver.quit()  # Fechar o navegador para liberar recursos

    return lista_de_imoveis

def salvar_excel(dataframe):
    dataframe.to_excel('imoveis_scrapping_thais_imobiliaria_venda.xlsx', index=False)

def main():
    num_clicks_input = input("Quantas vezes deseja clicar no botão 'Ver Mais'? ")
    num_clicks = int(num_clicks_input) if num_clicks_input else 311  # Usar 5 cliques padrão se nenhum valor for fornecido
    lista_de_imoveis = scrape_imoveis(num_clicks)

    df_imovel = pd.DataFrame(lista_de_imoveis, columns=['Título', 'Tipo', 'Link', 'Preço', 'Metro Quadrado', 'Quarto', 'Suite', 'Banheiro', 'Vaga'])

    # Limpeza de dados e conversão para números
    df_imovel['Preço'] = pd.to_numeric(df_imovel['Preço'], errors='coerce')
    df_imovel['Metro Quadrado'] = pd.to_numeric(df_imovel['Metro Quadrado'], errors='coerce')
    df_imovel['Quarto'] = pd.to_numeric(df_imovel['Quarto'], errors='coerce')
    df_imovel['Suite'] = pd.to_numeric(df_imovel['Suite'], errors='coerce')
    df_imovel['Banheiro'] = pd.to_numeric(df_imovel['Banheiro'], errors='coerce')
    df_imovel['Vaga'] = pd.to_numeric(df_imovel['Vaga'], errors='coerce')

    # Verificar valores nulos
    if df_imovel.isnull().values.any():
        print("Existem valores nulos no DataFrame. Lidar com eles conforme necessário.")

    df_imovel['M2'] = df_imovel['Preço'] / df_imovel['Metro Quadrado']
    
    # Substituir espaços em branco por 0
    df_imovel[['Quarto', 'Suite', 'Banheiro', 'Vaga', 'M2']] = df_imovel[['Quarto', 'Suite', 'Banheiro', 'Vaga', 'M2']].fillna(0)
    
    salvar_excel(df_imovel)

    print(df_imovel)

if __name__ == "__main__":
    main()


Clique 1
Clique 2
Clique 3
Clique 4
Clique 5
Clique 6
Clique 7
Clique 8
Clique 9
Clique 10
Clique 11
Clique 12
Clique 13
Clique 14
Clique 15
Clique 16
Clique 17
Clique 18
Clique 19
Clique 20
Clique 21
Clique 22
Clique 23
Clique 24
Clique 25
Clique 26
Clique 27
Clique 28
Clique 29
Clique 30
Clique 31
Clique 32
Clique 33
Clique 34
Clique 35
Clique 36
Clique 37
Clique 38
Clique 39
Clique 40
Clique 41
Clique 42
Clique 43
Clique 44
Clique 45
Clique 46
Clique 47
Clique 48
Clique 49
Clique 50
Clique 51
Clique 52
Clique 53
Clique 54
Clique 55
Clique 56
Clique 57
Clique 58
Clique 59
Clique 60
Clique 61
Clique 62
Clique 63
Clique 64
Clique 65
Clique 66
Clique 67
Clique 68
Clique 69
Clique 70
Clique 71
Clique 72
Clique 73
Clique 74
Clique 75
Clique 76
Clique 77
Clique 78
Clique 79
Clique 80
Clique 81
Clique 82
Clique 83
Clique 84
Clique 85
Clique 86
Clique 87
Clique 88
Clique 89
Clique 90
Clique 91
Clique 92
Clique 93
Clique 94
Clique 95
Clique 96
Clique 97
Clique 98
Clique 99
Clique 100
Clique 1

WebDriverException: Message: disconnected: received Inspector.detached event
  (Session info: chrome-headless-shell=124.0.6367.61)
Stacktrace:
	GetHandleVerifier [0x003FB133+46099]
	(No symbol) [0x0038C461]
	(No symbol) [0x00280C57]
	(No symbol) [0x0026E0E4]
	(No symbol) [0x0026D864]
	(No symbol) [0x0026D3FA]
	(No symbol) [0x0026D399]
	(No symbol) [0x0026B6E4]
	(No symbol) [0x0026BB1B]
	(No symbol) [0x0026A925]
	(No symbol) [0x00289542]
	(No symbol) [0x002B7A58]
	(No symbol) [0x002E0FFC]
	(No symbol) [0x002B7075]
	(No symbol) [0x002E1294]
	(No symbol) [0x002F9E15]
	(No symbol) [0x002E0D96]
	(No symbol) [0x002B595D]
	(No symbol) [0x002B651D]
	GetHandleVerifier [0x006B4BA3+2903683]
	GetHandleVerifier [0x006F261B+3156219]
	GetHandleVerifier [0x0049552B+677899]
	GetHandleVerifier [0x0049B6FC+702940]
	(No symbol) [0x00395A24]
	(No symbol) [0x00391A68]
	(No symbol) [0x00391BFC]
	(No symbol) [0x00384750]
	BaseThreadInitThunk [0x75C97BA9+25]
	RtlInitializeExceptionChain [0x775ABE3B+107]
	RtlCl